## 定义joint 并使用 ctrl 控制

In [ ]:
import os
import sys

if sys.platform == 'linux':
    os.environ['MUJOCO_GL'] = 'egl'
    # os.environ['MUJOCO_GL'] = 'osmesa'
elif sys.platform == 'darwin':
    os.environ['MUJOCO_GL'] = 'glfw'

import mujoco
import mediapy as media

# 创建XML模型字符串
drawer_xml = """
<mujoco model="drawer_demo">
    <compiler angle="radian" coordinate="local" eulerseq="XYZ"/>
    <option timestep="0.002" gravity="0 0 -9.81"/>
    
    <asset>
        <texture type="skybox" builtin="gradient" rgb1="0.3 0.5 0.7" rgb2="0 0 0" width="512" height="512"/>
        <texture name="wood" type="cube" builtin="flat" rgb1="0.6 0.3 0.1" width="100" height="100"/>
        <material name="wood" texture="wood" texrepeat="1 1" texuniform="true" reflectance="0.2"/>
        <material name="drawer_metal" rgba="0.8 0.8 0.9 1" shininess="0.8" reflectance="0.5"/>
    </asset>
    
    <worldbody>
        <!-- 相机 -->
        <camera pos="0.013 1.337 2.105" xyaxes="-1.000 -0.003 -0.000 0.003 -0.781 0.624"/>
    
        <!-- 光源 -->
        <light pos="0 0 3" dir="0 0 -1" diffuse="0.7 0.7 0.7" specular="0.3 0.3 0.3"/>
        <light pos="0 3 0" dir="0 -1 0" diffuse="0.7 0.7 0.7" specular="0.3 0.3 0.3"/>
        
        <!-- 地板 -->
        <geom name="floor" type="plane" size="2 2 0.01" rgba="0.5 0.5 0.5 1"/>
        
        <!-- 柜子主体 - 固定在世界中 -->
        <body name="cabinet" pos="0 0 0.4">
            <!-- 柜子底部 -->
            <geom name="cabinet_bottom" type="box" size="0.4 0.3 0.02" pos="0 0 0" material="wood"/>
            <!-- 柜子顶部 -->
            <geom name="cabinet_top" type="box" size="0.4 0.3 0.02" pos="0 0 0.4" material="wood"/>
            <!-- 柜子左侧 -->
            <geom name="cabinet_left" type="box" size="0.02 0.3 0.2" pos="-0.38 0 0.2" material="wood"/>
            <!-- 柜子右侧 -->
            <geom name="cabinet_right" type="box" size="0.02 0.3 0.2" pos="0.38 0 0.2" material="wood"/>
            <!-- 柜子背面 -->
            <geom name="cabinet_back" type="box" size="0.4 0.02 0.2" pos="0 -0.28 0.2" material="wood"/>
            
            <!-- 抽屉 - 使用prismatic joint -->
            <body name="drawer" pos="0 0 0.1">
                <!-- 直线关节，沿Y轴移动 -->
                <joint name="drawer_slide" type="slide" axis="0 1 0" range="0 0.25" damping="0.1"/>
                
                <!-- 抽屉底部 -->
                <geom name="drawer_bottom" type="box" size="0.35 0.25 0.02" pos="0 0 0" material="wood"/>
                <!-- 抽屉前面 -->
                <geom name="drawer_front" type="box" size="0.35 0.02 0.08" pos="0 0.23 0.08" material="wood"/>
                <!-- 抽屉背面 -->
                <geom name="drawer_back" type="box" size="0.35 0.02 0.08" pos="0 -0.23 0.08" material="wood"/>
                <!-- 抽屉左侧 -->
                <geom name="drawer_left" type="box" size="0.02 0.25 0.08" pos="-0.33 0 0.08" material="wood"/>
                <!-- 抽屉右侧 -->
                <geom name="drawer_right" type="box" size="0.02 0.25 0.08" pos="0.33 0 0.08" material="wood"/>
                
                <!-- 抽屉把手 -->
                <geom name="handle" type="cylinder" size="0.02 0.1" pos="0 0.25 0.08" euler="1.57 0 0" material="drawer_metal"/>
            </body>
        </body>
    </worldbody>

    <actuator>
        <!-- 控制抽屉位置的执行器 -->
        <position name="drawer_control" joint="drawer_slide" kp="100"/>
    </actuator>

</mujoco>
"""

# 从XML字符串加载模型
mj_model = mujoco.MjModel.from_xml_string(drawer_xml)
mj_data = mujoco.MjData(mj_model)

# 初始化渲染器
renderer = mujoco.Renderer(mj_model, height=480, width=640)

# 展示抽屉操作动画
def show_drawer_animation():
    # 收集图像帧
    frames = []
    
    # 初始状态 - 抽屉关闭
    mj_data.ctrl[0] = 0.0
    for _ in range(20):  # 模拟一些时间步
        mujoco.mj_step(mj_model, mj_data)
    
    # 添加初始帧
    renderer.update_scene(mj_data, 0)
    frames.append(renderer.render().copy())
    
    # 打开抽屉
    mj_data.ctrl[0] = 1
    for i in range(30):
        # 执行多步模拟以使抽屉位置稳定
        for _ in range(5):
            mujoco.mj_step(mj_model, mj_data)
        
        # 渲染并存储帧
        renderer.update_scene(mj_data, 0)
        frames.append(renderer.render().copy())
    
    # 保持抽屉打开一段时间
    for _ in range(10):
        for _ in range(5):
            mujoco.mj_step(mj_model, mj_data)
        renderer.update_scene(mj_data, 0)
        frames.append(renderer.render().copy())
    
    # 关闭抽屉
    mj_data.ctrl[0] = -1
    for i in range(30):
        
        # 执行多步模拟
        for _ in range(5):
            mujoco.mj_step(mj_model, mj_data)
        
        # 渲染并存储帧
        renderer.update_scene(mj_data, 0)
        frames.append(renderer.render().copy())
    
    # 使用mediapy显示动画
    media.show_video(frames, fps=15)
    
    return frames

# 渲染单张图像
mujoco.mj_forward(mj_model, mj_data)
renderer.update_scene(mj_data, 0)
img = renderer.render()
media.show_image(img)

# 运行动画
frames = show_drawer_animation()

## 定义joint 直接设置 joint 位置

In [ ]:
%env MUJOCO_GL=egl

import mujoco
import mediapy as media

# 创建XML模型字符串
drawer_xml = """
<mujoco model="drawer_demo">
    <compiler angle="radian" coordinate="local" eulerseq="XYZ"/>
    <option timestep="0.002" gravity="0 0 -9.81"/>
    
    <asset>
        <texture type="skybox" builtin="gradient" rgb1="0.3 0.5 0.7" rgb2="0 0 0" width="512" height="512"/>
        <texture name="wood" type="cube" builtin="flat" rgb1="0.6 0.3 0.1" width="100" height="100"/>
        <material name="wood" texture="wood" texrepeat="1 1" texuniform="true" reflectance="0.2"/>
        <material name="drawer_metal" rgba="0.8 0.8 0.9 1" shininess="0.8" reflectance="0.5"/>
    </asset>
    
    <worldbody>
        <!-- 相机 -->
        <camera pos="0.013 1.337 2.105" xyaxes="-1.000 -0.003 -0.000 0.003 -0.781 0.624"/>
    
        <!-- 光源 -->
        <light pos="0 0 3" dir="0 0 -1" diffuse="0.7 0.7 0.7" specular="0.3 0.3 0.3"/>
        <light pos="0 3 0" dir="0 -1 0" diffuse="0.7 0.7 0.7" specular="0.3 0.3 0.3"/>
        
        <!-- 地板 -->
        <geom name="floor" type="plane" size="2 2 0.01" rgba="0.5 0.5 0.5 1"/>
        
        <!-- 柜子主体 - 固定在世界中 -->
        <body name="cabinet" pos="0 0 0.4">
            <!-- 柜子底部 -->
            <geom name="cabinet_bottom" type="box" size="0.4 0.3 0.02" pos="0 0 0" material="wood"/>
            <!-- 柜子顶部 -->
            <geom name="cabinet_top" type="box" size="0.4 0.3 0.02" pos="0 0 0.4" material="wood"/>
            <!-- 柜子左侧 -->
            <geom name="cabinet_left" type="box" size="0.02 0.3 0.2" pos="-0.38 0 0.2" material="wood"/>
            <!-- 柜子右侧 -->
            <geom name="cabinet_right" type="box" size="0.02 0.3 0.2" pos="0.38 0 0.2" material="wood"/>
            <!-- 柜子背面 -->
            <geom name="cabinet_back" type="box" size="0.4 0.02 0.2" pos="0 -0.28 0.2" material="wood"/>
            
            <!-- 抽屉 - 使用prismatic joint -->
            <body name="drawer" pos="0 0 0.1">
                <!-- 直线关节，沿Y轴移动 -->
                <joint name="drawer_slide" type="slide" axis="0 1 0" range="0 0.25" damping="0.1"/>
                
                <!-- 抽屉底部 -->
                <geom name="drawer_bottom" type="box" size="0.35 0.25 0.02" pos="0 0 0" material="wood"/>
                <!-- 抽屉前面 -->
                <geom name="drawer_front" type="box" size="0.35 0.02 0.08" pos="0 0.23 0.08" material="wood"/>
                <!-- 抽屉背面 -->
                <geom name="drawer_back" type="box" size="0.35 0.02 0.08" pos="0 -0.23 0.08" material="wood"/>
                <!-- 抽屉左侧 -->
                <geom name="drawer_left" type="box" size="0.02 0.25 0.08" pos="-0.33 0 0.08" material="wood"/>
                <!-- 抽屉右侧 -->
                <geom name="drawer_right" type="box" size="0.02 0.25 0.08" pos="0.33 0 0.08" material="wood"/>
                
                <!-- 抽屉把手 -->
                <geom name="handle" type="cylinder" size="0.02 0.1" pos="0 0.25 0.08" euler="1.57 0 0" material="drawer_metal"/>
            </body>
        </body>
    </worldbody>

    <!-- 注意这里删掉了 控制抽屉位置的执行器 -->
       
</mujoco>
"""

# 从XML字符串加载模型
mj_model = mujoco.MjModel.from_xml_string(drawer_xml)
mj_data = mujoco.MjData(mj_model)

# 获取抽屉关节ID 主要需要通过关节名字来获取id 在xml中需要给joint一个name
qid = mj_model.joint("drawer_slide").id

# 初始化渲染器
renderer = mujoco.Renderer(mj_model, height=480, width=640)

# 展示抽屉操作动画
def show_drawer_animation():
    # 收集图像帧
    frames = []
    
    # 初始状态 - 抽屉关闭
    mj_data.qpos[qid] = 0.0
    for _ in range(20):  # 模拟一些时间步
        mujoco.mj_step(mj_model, mj_data)
    
    # 添加初始帧
    renderer.update_scene(mj_data, 0)
    frames.append(renderer.render().copy())
    
    for i in range(30):
        # 打开抽屉
        target_pos = i * 0.25 / 30  # 从0到0.25
        ###################################################################
        # 这里直接修改了qpos，而不是ctrl
        mj_data.qpos[qid] = target_pos
        ###################################################################

        # 执行多步模拟以使抽屉位置稳定
        for _ in range(5):
            mujoco.mj_step(mj_model, mj_data)
        
        # 渲染并存储帧
        renderer.update_scene(mj_data, 0)
        frames.append(renderer.render().copy())
    
    # 保持抽屉打开一段时间
    for _ in range(10):
        for _ in range(5):
            mujoco.mj_step(mj_model, mj_data)
        renderer.update_scene(mj_data, 0)
        frames.append(renderer.render().copy())
    
    # 关闭抽屉
    for i in range(30):
        target_pos = 0.25 - i * 0.25 / 30  # 从0.25到0
        mj_data.qpos[qid] = target_pos

        # 执行多步模拟
        for _ in range(5):
            mujoco.mj_step(mj_model, mj_data)
        
        # 渲染并存储帧
        renderer.update_scene(mj_data, 0)
        frames.append(renderer.render().copy())
    
    # 使用mediapy显示动画
    media.show_video(frames, fps=15)
    
    return frames

# 渲染单张图像
mujoco.mj_forward(mj_model, mj_data)
renderer.update_scene(mj_data, 0)
img = renderer.render()
media.show_image(img)

# 运行动画
frames = show_drawer_animation()

## 不设置joint 直接修改body pos 来设置抽屉位置

In [ ]:
%env MUJOCO_GL=egl

import mujoco
import mediapy as media

# 创建XML模型字符串
drawer_xml = """
<mujoco model="drawer_demo">
    <compiler angle="radian" coordinate="local" eulerseq="XYZ"/>
    <option timestep="0.002" gravity="0 0 -9.81"/>
    
    <asset>
        <texture type="skybox" builtin="gradient" rgb1="0.3 0.5 0.7" rgb2="0 0 0" width="512" height="512"/>
        <texture name="wood" type="cube" builtin="flat" rgb1="0.6 0.3 0.1" width="100" height="100"/>
        <material name="wood" texture="wood" texrepeat="1 1" texuniform="true" reflectance="0.2"/>
        <material name="drawer_metal" rgba="0.8 0.8 0.9 1" shininess="0.8" reflectance="0.5"/>
    </asset>
    
    <worldbody>
        <!-- 相机 -->
        <camera pos="0.013 1.337 2.105" xyaxes="-1.000 -0.003 -0.000 0.003 -0.781 0.624"/>
    
        <!-- 光源 -->
        <light pos="0 0 3" dir="0 0 -1" diffuse="0.7 0.7 0.7" specular="0.3 0.3 0.3"/>
        <light pos="0 3 0" dir="0 -1 0" diffuse="0.7 0.7 0.7" specular="0.3 0.3 0.3"/>
        
        <!-- 地板 -->
        <geom name="floor" type="plane" size="2 2 0.01" rgba="0.5 0.5 0.5 1"/>
        
        <!-- 柜子主体 - 固定在世界中 -->
        <body name="cabinet" pos="0 0 0.4">
            <!-- 柜子底部 -->
            <geom name="cabinet_bottom" type="box" size="0.4 0.3 0.02" pos="0 0 0" material="wood"/>
            <!-- 柜子顶部 -->
            <geom name="cabinet_top" type="box" size="0.4 0.3 0.02" pos="0 0 0.4" material="wood"/>
            <!-- 柜子左侧 -->
            <geom name="cabinet_left" type="box" size="0.02 0.3 0.2" pos="-0.38 0 0.2" material="wood"/>
            <!-- 柜子右侧 -->
            <geom name="cabinet_right" type="box" size="0.02 0.3 0.2" pos="0.38 0 0.2" material="wood"/>
            <!-- 柜子背面 -->
            <geom name="cabinet_back" type="box" size="0.4 0.02 0.2" pos="0 -0.28 0.2" material="wood"/>
            
            <!-- 抽屉 - 使用prismatic joint -->
            <body name="drawer" pos="0 0 0.1">
                
                <!-- 注意这里删掉了 沿着y轴移动的直线关节 -->
               
                <!-- 抽屉底部 -->
                <geom name="drawer_bottom" type="box" size="0.35 0.25 0.02" pos="0 0 0" material="wood"/>
                <!-- 抽屉前面 -->
                <geom name="drawer_front" type="box" size="0.35 0.02 0.08" pos="0 0.23 0.08" material="wood"/>
                <!-- 抽屉背面 -->
                <geom name="drawer_back" type="box" size="0.35 0.02 0.08" pos="0 -0.23 0.08" material="wood"/>
                <!-- 抽屉左侧 -->
                <geom name="drawer_left" type="box" size="0.02 0.25 0.08" pos="-0.33 0 0.08" material="wood"/>
                <!-- 抽屉右侧 -->
                <geom name="drawer_right" type="box" size="0.02 0.25 0.08" pos="0.33 0 0.08" material="wood"/>
                
                <!-- 抽屉把手 -->
                <geom name="handle" type="cylinder" size="0.02 0.1" pos="0 0.25 0.08" euler="1.57 0 0" material="drawer_metal"/>
            </body>
        </body>
    </worldbody>

</mujoco>
"""

# 从XML字符串加载模型
mj_model = mujoco.MjModel.from_xml_string(drawer_xml)
mj_data = mujoco.MjData(mj_model)

# 渲染单张图像
mujoco.mj_forward(mj_model, mj_data)
renderer.update_scene(mj_data, 0)
img = renderer.render()
media.show_image(img)

# 查看 drawer 的状态
# 注意这里的位置是相对于父节点的位置
print("local position    : ", mj_model.body("drawer").pos)
print("local orientation : ", mj_model.body("drawer").quat) # 四元数 顺序为 w x y z

# 查看其父节点的名称
# body 可以通过 id 和 name 来获取
print(mj_model.body(mj_model.body("drawer").rootid).name)

# 查看 drawer 的全局位置
# 通过 mj_data 获取
# 注意在这里的位置是全局位置，也就是相对于世界坐标系的位置
# 需要先运行 mj_forward 或者 mj_step 更新 mj_data 的数据
print("global position    : ", mj_data.xpos[mj_data.body("drawer").id])
print("global orientation : ", mj_data.xquat[mj_data.body("drawer").id])

# 下面的方式是一样的：
# print("global position    : ", mj_data.body("drawer").xpos)
# print("global orientation : ", mj_data.body("drawer").xquat)

# 抽屉是沿着y轴移动的
# 因此需将抽屉在其父节点坐标系下 沿着 y 轴移动
# drawer 是固定在 cabinet 上的(没有joint自由度) ，这种情况下不能直接修改 mj_data里的数据,需要通过修改 mj_model里的数据

# 展示抽屉操作动画
def show_drawer_animation():
    # 收集图像帧
    frames = []
    
    # 初始状态 - 抽屉关闭
    mj_model.body("drawer").pos[1] = 0
    for _ in range(20):  # 模拟一些时间步
        mujoco.mj_step(mj_model, mj_data)
    
    # 添加初始帧
    renderer.update_scene(mj_data, 0)
    frames.append(renderer.render().copy())
    
    for i in range(30):
        # 打开抽屉
        target_pos = i * 0.25 / 30  # 从0到0.25
        ###################################################################
        # 这里直接修改了mj_model.body 的y轴坐标
        mj_model.body("drawer").pos[1] = target_pos
        ###################################################################

        # 执行多步模拟以使抽屉位置稳定
        for _ in range(5):
            mujoco.mj_step(mj_model, mj_data)
        
        # 渲染并存储帧
        renderer.update_scene(mj_data, 0)
        frames.append(renderer.render().copy())
    
    # 保持抽屉打开一段时间
    for _ in range(10):
        for _ in range(5):
            mujoco.mj_step(mj_model, mj_data)
        renderer.update_scene(mj_data, 0)
        frames.append(renderer.render().copy())
    
    # 关闭抽屉
    for i in range(30):
        target_pos = 0.25 - i * 0.25 / 30  # 从0.25到0
        mj_model.body("drawer").pos[1] = target_pos

        # 执行多步模拟
        for _ in range(5):
            mujoco.mj_step(mj_model, mj_data)
        
        # 渲染并存储帧
        renderer.update_scene(mj_data, 0)
        frames.append(renderer.render().copy())
    
    # 使用mediapy显示动画
    media.show_video(frames, fps=15)
    
    return frames

frames = show_drawer_animation()